# LeNet

LeNet is a convolutional neural network architecture introduced by Yann LeCun in 1998. It was designed for handwritten digit recognition and laid the foundation for many modern CNN architectures.

LeNet-5 consists of the following layers:

- Input layer (32x32 grayscale image)

- Convolutional layer (6 feature maps, 5x5 kernel)

- Average pooling layer (2x2)

- Convolutional layer (16 feature maps, 5x5 kernel)

- Average pooling layer (2x2)

- Fully connected layer (120 units)

- Fully connected layer (84 units)

- Output layer (10 units, one for each digit)


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        self.avgpool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # Updated
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.avgpool(torch.relu(self.conv1(x)))
        x = self.avgpool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 6 * 6)  # This remains unchanged
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


## Hyperparameters

In [3]:
batch_size = 64
learning_rate = 0.001
num_epochs = 10

## Load and preprocess the MNIST dataset

In [4]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Initialize the model, loss function, and optimizer

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Training loop

In [6]:
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}")


Epoch 1/10, Batch 0/938, Loss: 2.3211
Epoch 1/10, Batch 100/938, Loss: 0.4877
Epoch 1/10, Batch 200/938, Loss: 0.3070
Epoch 1/10, Batch 300/938, Loss: 0.3086
Epoch 1/10, Batch 400/938, Loss: 0.1336
Epoch 1/10, Batch 500/938, Loss: 0.1447
Epoch 1/10, Batch 600/938, Loss: 0.1581
Epoch 1/10, Batch 700/938, Loss: 0.0949
Epoch 1/10, Batch 800/938, Loss: 0.0866
Epoch 1/10, Batch 900/938, Loss: 0.1418
Epoch 2/10, Batch 0/938, Loss: 0.1562
Epoch 2/10, Batch 100/938, Loss: 0.0506
Epoch 2/10, Batch 200/938, Loss: 0.1366
Epoch 2/10, Batch 300/938, Loss: 0.0910
Epoch 2/10, Batch 400/938, Loss: 0.0578
Epoch 2/10, Batch 500/938, Loss: 0.0416
Epoch 2/10, Batch 600/938, Loss: 0.1034
Epoch 2/10, Batch 700/938, Loss: 0.1061
Epoch 2/10, Batch 800/938, Loss: 0.0342
Epoch 2/10, Batch 900/938, Loss: 0.0132
Epoch 3/10, Batch 0/938, Loss: 0.0419
Epoch 3/10, Batch 100/938, Loss: 0.0376
Epoch 3/10, Batch 200/938, Loss: 0.0778
Epoch 3/10, Batch 300/938, Loss: 0.0166
Epoch 3/10, Batch 400/938, Loss: 0.1417
Epoch 

## Evaluation

In [7]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")

Accuracy on test set: 98.98%
